In [2]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torch import optim
import timm
import os
from torch.nn import functional as F
from PIL import Image

c:\anaconda\envs\torch-2.0\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
backbone = timm.create_model(
                "vit_base_patch16_224",
                pretrained=True,
            )
original_backbone = timm.create_model("vit_base_patch16_224",
                pretrained=True,
            )

In [4]:
class Classifier(torch.nn.Module):
    def __init__(self, input_dim, nb_classes, *args, **kwargs) -> None:
        super(Classifier, self).__init__(*args, **kwargs)
        self.fc = torch.nn.Linear(input_dim, nb_classes)
    def forward(self, x):
        return self.fc(x)
    

In [5]:
class Cifar100():
    def __init__(self):
        self.train_trsf = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=(0.05, 1.0), ratio=(3. / 4., 4. / 3.)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ToTensor()
        ])
        self.test_trsf = transforms.Compose([
            transforms.Resize(int((256 / 224) * 224), interpolation=3),
            transforms.CenterCrop(224),
            transforms.ToTensor()
        ])
        self.class_order = np.arange(100).tolist()
        self.train_dataset = datasets.cifar.CIFAR100("./data", train=True, download=True)
        self.test_dataset = datasets.cifar.CIFAR100("./data", train=False, download=True)
        self.train_data, self.train_targets = self.train_dataset.data, np.array(
            self.train_dataset.targets
        )
        self.test_data, self.test_targets = self.test_dataset.data, np.array(
            self.test_dataset.targets
        )

In [6]:
class Cifar10():
    def __init__(self):
        self.train_trsf = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=(0.05, 1.0), ratio=(3. / 4., 4. / 3.)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ToTensor()
        ])
        self.test_trsf = transforms.Compose([
            transforms.Resize(int((256 / 224) * 224), interpolation=3),
            transforms.CenterCrop(224),
            transforms.ToTensor()
        ])
        self.train_dataset = datasets.cifar.CIFAR10("./data", train=True, download=True)
        self.test_dataset = datasets.cifar.CIFAR10("./data", train=False, download=True)
        self.train_data, self.train_targets = self.train_dataset.data, np.array(
            self.train_dataset.targets
        )
        self.test_data, self.test_targets = self.test_dataset.data, np.array(
            self.test_dataset.targets
        )

In [7]:
class DummyDataset(Dataset):
    def __init__(self, images, labels, trsf) -> None:
        super().__init__()
        self.images = images
        self.labels = labels
        self.trsf = trsf
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = self.trsf(Image.fromarray(self.images[idx]))
        label = self.labels[idx]
        return idx, image, label

In [8]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
dataset = Cifar10()
train_idx = np.where(np.logical_and(dataset.train_targets >= 0, dataset.train_targets < 2))[0]
test_idx = np.where(np.logical_and(dataset.test_targets >= 2, dataset.test_targets < 4))[0]

Files already downloaded and verified
Files already downloaded and verified


In [10]:
train_dataset = DummyDataset(dataset.train_data[train_idx], dataset.train_targets[train_idx], dataset.train_trsf)
test_dataset = DummyDataset(dataset.test_data[test_idx], dataset.test_targets[test_idx], dataset.test_trsf)
train_dataloader = DataLoader(train_dataset, 8, True)
test_dataloader = DataLoader(test_dataset, 8, True)

In [11]:
original_backbone.eval()
for param in original_backbone.parameters():
    param.requires_grad = False
    
input_dim = original_backbone.head.in_features
nb_classes = 10
classifier = Classifier(input_dim, nb_classes)
original_backbone.head = classifier

In [12]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(original_backbone.head.parameters(), lr=0.001)

In [13]:
epochs = 2
original_backbone.cuda()
criterion.cuda()
for e in range(epochs):
    original_backbone.train()
    for idx, x, y in train_dataloader:
        x, y = x.cuda(), y.cuda()
        optimizer.zero_grad()
        y_pred = original_backbone(x)
        loss = criterion(y_pred, y.long())
        loss.backward()
        optimizer.step()
        print('Epoch [{}/{}], Loss: {:.4f}'.format(e + 1, epochs, loss.item()))

Epoch [1/2], Loss: 4.1946
Epoch [1/2], Loss: 1.7761
Epoch [1/2], Loss: 1.1790
Epoch [1/2], Loss: 0.4658
Epoch [1/2], Loss: 1.2176
Epoch [1/2], Loss: 0.9043
Epoch [1/2], Loss: 0.1730
Epoch [1/2], Loss: 0.1920
Epoch [1/2], Loss: 0.8435
Epoch [1/2], Loss: 0.1193
Epoch [1/2], Loss: 0.4580
Epoch [1/2], Loss: 0.4158
Epoch [1/2], Loss: 0.3906
Epoch [1/2], Loss: 0.7593
Epoch [1/2], Loss: 0.1393
Epoch [1/2], Loss: 0.5850
Epoch [1/2], Loss: 0.3504
Epoch [1/2], Loss: 0.2481
Epoch [1/2], Loss: 0.1653
Epoch [1/2], Loss: 0.1890
Epoch [1/2], Loss: 0.0167
Epoch [1/2], Loss: 0.1377
Epoch [1/2], Loss: 0.0331
Epoch [1/2], Loss: 0.0334
Epoch [1/2], Loss: 0.1111
Epoch [1/2], Loss: 0.5812
Epoch [1/2], Loss: 0.1832
Epoch [1/2], Loss: 0.0334
Epoch [1/2], Loss: 0.7158
Epoch [1/2], Loss: 0.7715
Epoch [1/2], Loss: 0.9059
Epoch [1/2], Loss: 0.2323
Epoch [1/2], Loss: 0.5980
Epoch [1/2], Loss: 0.0609
Epoch [1/2], Loss: 0.0823
Epoch [1/2], Loss: 0.0400
Epoch [1/2], Loss: 0.0177
Epoch [1/2], Loss: 0.0061
Epoch [1/2],

In [14]:
original_backbone.eval()
predictions = []  
ground_truths = []  
  
with torch.no_grad():  # 关闭梯度计算，以节省内存并加速测试过程  
    for idx, images, labels in test_dataloader:  
        images, labels = images.to(device), labels.to(device)
        outputs = original_backbone(images)  # 通过模型传递输入数据并获取预测结果  
        _, predicted = torch.max(outputs.data, 1)  # 对预测结果进行argmax操作，获取预测的类别索引  
        predictions.extend(predicted.tolist())  # 将预测结果添加到预测列表中  
        ground_truths.extend(labels.tolist())  # 将实际标签添加到实际标签列表中  
  
# 计算准确率作为评估指标  
accuracy = np.mean(np.array(predictions) == np.array(ground_truths)) * 100  # 计算准确率（百分比）  
print('Test Accuracy: {:.2f}%'.format(accuracy))  # 打印测试准确率结果


Test Accuracy: 0.05%


In [15]:
train_idx = np.where(np.logical_and(dataset.train_targets >= 2, dataset.train_targets < 4))[0]
test_idx = np.where(np.logical_and(dataset.test_targets >= 2, dataset.test_targets < 4))[0]
train_dataset = DummyDataset(dataset.train_data[train_idx], dataset.train_targets[train_idx], dataset.train_trsf)
test_dataset = DummyDataset(dataset.test_data[test_idx], dataset.test_targets[test_idx], dataset.test_trsf)
train_dataloader = DataLoader(train_dataset, 8, True)
test_dataloader = DataLoader(test_dataset, 8, True)

In [16]:
epochs = 2
for e in range(epochs):
    original_backbone.train()
    for idx, x, y in train_dataloader:
        x, y = x.cuda(), y.cuda()
        optimizer.zero_grad()
        y_pred = original_backbone(x)
        loss = criterion(y_pred, y.long())
        loss.backward()
        optimizer.step()
        print('Epoch [{}/{}], Loss: {:.4f}'.format(e + 1, epochs, loss.item()))

Epoch [1/2], Loss: 14.0848
Epoch [1/2], Loss: 11.9344
Epoch [1/2], Loss: 4.9162
Epoch [1/2], Loss: 3.4870
Epoch [1/2], Loss: 1.0625
Epoch [1/2], Loss: 2.0590
Epoch [1/2], Loss: 0.3402
Epoch [1/2], Loss: 0.5166
Epoch [1/2], Loss: 0.1033
Epoch [1/2], Loss: 1.0887
Epoch [1/2], Loss: 1.7765
Epoch [1/2], Loss: 0.0186
Epoch [1/2], Loss: 0.6663
Epoch [1/2], Loss: 0.6290
Epoch [1/2], Loss: 0.4383
Epoch [1/2], Loss: 0.0810
Epoch [1/2], Loss: 0.0387
Epoch [1/2], Loss: 0.2064
Epoch [1/2], Loss: 0.1173
Epoch [1/2], Loss: 1.5153
Epoch [1/2], Loss: 0.4983
Epoch [1/2], Loss: 0.0011
Epoch [1/2], Loss: 0.0979
Epoch [1/2], Loss: 0.1784
Epoch [1/2], Loss: 0.0786
Epoch [1/2], Loss: 0.0000
Epoch [1/2], Loss: 0.7008
Epoch [1/2], Loss: 1.0472
Epoch [1/2], Loss: 0.0065
Epoch [1/2], Loss: 0.9127
Epoch [1/2], Loss: 1.6966
Epoch [1/2], Loss: 1.1861
Epoch [1/2], Loss: 0.0975
Epoch [1/2], Loss: 0.0038
Epoch [1/2], Loss: 0.0501
Epoch [1/2], Loss: 0.0013
Epoch [1/2], Loss: 0.4203
Epoch [1/2], Loss: 1.0869
Epoch [1/2

In [17]:
original_backbone.eval()
predictions = []
ground_truths = []
  
with torch.no_grad():  # 关闭梯度计算，以节省内存并加速测试过程  
    for idx, images, labels in test_dataloader:  
        images, labels = images.to(device), labels.to(device)
        print(type(labels))
        outputs = original_backbone(images)  # 通过模型传递输入数据并获取预测结果
        p = outputs.softmax(dim=1)
        confidence, predicted = torch.max(p, 1)  # 对预测结果进行argmax操作，获取预测的类别索引  
        confidence = torch.where(confidence > 0.9, 1, 0)
        i = confidence.nonzero().view(-1)
        x_unlabeled = images[i]
        pseudo_label = predicted[i]
        i = confidence.nonzero().view(-1)
        x_unlabeled = images[i]
        pseudo_label = predicted[i]
        
        # predictions.extend(predicted.tolist())  # 将预测结果添加到预测列表中  
        # ground_truths.extend(labels.tolist())  # 将实际标签添加到实际标签列表中 
        break
    
    
  
# 计算准确率作为评估指标  
# accuracy = np.mean(np.array(predictions) == np.array(ground_truths)) * 100  # 计算准确率（百分比）  
# print('Test Accuracy: {:.2f}%'.format(accuracy))  # 打印测试准确率结果

<class 'torch.Tensor'>


In [18]:
print(type(dataset.train_data))

<class 'numpy.ndarray'>
